In [1]:
import os
import re
import datetime
import math
from enum import Enum
import numpy as np
import tensorflow as tf
import keras as k
import scipy.io
import h5py
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
#from data_handling import CppDataExtraction,EigenData,XyDataFormat,MlModelDataFormat,PlotType,MlPlots

In [3]:
class EigenData(object):
    '''Represents the original experiment data.'''

    def __init__(self):
        self._eigenworms = None

    def get_eigenworms(self, eigen_path):
        '''
        Loads the matlab files from the original experiment and parses them
        into expected numpy formats.

        Eigenworms are stored as 100 'angles' at equidistributed coordinates down the body.
        
        The original format of the Stephens data eigenmodes.
        '''
        # Load the matlab files into numpy arrays
        # 100*100
        eigenworms = scipy.io.loadmat(eigen_path)
        self._eigenworms = eigenworms["EigenWorms"].transpose()
        return self._eigenworms

    def get_footage(self, footage_path):
        '''
        Loads the matlab files from the original experiment and parses them
        into expected numpy formats.

        Footage is stored as coefficients with respect to the eigenworm basis.
        
        Format is a cell array of 12 cells, each containing an Nx5 matrix, where a row is a time point, and each eigenmode has a column.
        
        experiment coefficient values
        
        The original format of the Stephens data eigenvalues
        '''
        # 12 * 5 *33600
        f = h5py.File(footage_path, 'r')
        footage = {}
        for k, v in f.items():
            if k != 'tr':
                for k2, v2 in v.items():
                    if v2.shape == (5, 33600) or v2.shape == (6, 33600):
                        footage[k2] = np.array(v2)
        return footage

    def reconstruct(self, coefficients):
        '''
        Reconstruct multiple postures from basis coefficients to angles.
        '''
        n_basis_required =  coefficients.shape[0]
        print(self._eigenworms[0:n_basis_required, :].transpose().shape,coefficients.shape)
        return self._eigenworms[0:n_basis_required, :].transpose() @ coefficients

In [4]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8679128990624087799
xla_global_id: -1
]


In [5]:
data = EigenData()
eigenworms=data.get_eigenworms('EigenWorms.mat')
footage = data.get_footage('20150814-All-PNAS2011-DataStitched .mat')

first_5_eigenworms = eigenworms[0:5,:]
eig_worm_0 = eigenworms[0,:]
eig_worm_1 = eigenworms[1,:]
eig_worm_2 = eigenworms[2,:]
eig_worm_3 = eigenworms[3,:]
eig_worm_4 = eigenworms[4,:]

for k in footage.keys():
    print(k, footage[k].shape)
    r = data.reconstruct(footage[k])
    print('-->', r.shape)

b (5, 33600)
(100, 5) (5, 33600)
--> (100, 33600)
c (5, 33600)
(100, 5) (5, 33600)
--> (100, 33600)
d (5, 33600)
(100, 5) (5, 33600)
--> (100, 33600)
e (5, 33600)
(100, 5) (5, 33600)
--> (100, 33600)
f (5, 33600)
(100, 5) (5, 33600)
--> (100, 33600)
g (6, 33600)
(100, 6) (6, 33600)
--> (100, 33600)
h (5, 33600)
(100, 5) (5, 33600)
--> (100, 33600)
i (6, 33600)
(100, 6) (6, 33600)
--> (100, 33600)
j (5, 33600)
(100, 5) (5, 33600)
--> (100, 33600)
k (5, 33600)
(100, 5) (5, 33600)
--> (100, 33600)
l (5, 33600)
(100, 5) (5, 33600)
--> (100, 33600)
m (5, 33600)
(100, 5) (5, 33600)
--> (100, 33600)


In [5]:
class CppDataExtraction(object):

    _nsre = re.compile('([0-9]+)')

    def __init__(self):
        pass

    @staticmethod    
    def natural_sort_key(s):
        return [int(text) if text.isdigit() else text.lower()
                for text in re.split(CppDataExtraction._nsre, s)]   

    @staticmethod
    def get_list_of_files(path):
        #TODO: Currently only setup for 1 job
        # loop through all the files in the job
        list_of_files = []
        for subdir, dirs, files in os.walk(path):
            for file in files:
                file_path = os.path.join(subdir, file)
                list_of_files.append(file_path)
        list_of_files.sort(key=CppDataExtraction.natural_sort_key)
        return list_of_files

    @staticmethod
    def get_list_of_points(list_of_files):
        list_of_data_points = []
        for file in list_of_files:
            f = open(file)
            lines = f.readlines()
            datapoints = np.array([x.split(',') for x in lines], dtype=np.float64)
            list_of_data_points.append(datapoints)
        return list_of_data_points

In [10]:
 rootdir_curvatures = 'trained-data'
list_of_curve_files = CppDataExtraction.get_list_of_files(rootdir_curvatures)
print([file[-9:-4] for file in list_of_curve_files])
#Load in the muscle curvatures
rootdir_muscle = 'eigenworm-drive-data-cp'
'''eigenworm-drive-data-cp
Contains the original Stephens experiment coefficient values for each of the 12 worms.'''
list_of_muscle_files = CppDataExtraction.get_list_of_files(rootdir_muscle)
# Remove the TEST file
list_of_muscle_files = list_of_muscle_files[:-1]
# Remove the zeroth file (TODO: NOT CURRENTLY PROCESSED)
list_of_muscle_files = list_of_muscle_files[1:]
print([file[-12:-4] for file in list_of_muscle_files])
list_of_curve_points = CppDataExtraction.get_list_of_points(list_of_curve_files)
print(([curve.shape for curve in list_of_curve_points]))
list_of_muscle_points = CppDataExtraction.get_list_of_points(list_of_muscle_files)
print(([muscle.shape for muscle in list_of_muscle_points]))

['worm1', 'worm2', 'worm3', 'worm4', 'worm5', 'worm6', 'worm7', 'worm8', 'worm9', 'orm10', 'orm11']
['eigvals1', 'eigvals2', 'eigvals3', 'eigvals4', 'eigvals5', 'eigvals6', 'eigvals7', 'eigvals8', 'eigvals9', 'igvals10', 'igvals11']
[(29871, 5), (31844, 5), (31862, 5), (32980, 5), (33095, 5), (33166, 5), (31316, 5), (33538, 5), (33457, 5), (33313, 5), (28494, 5)]
[(29871, 5), (31844, 5), (31862, 5), (32980, 5), (33095, 5), (33166, 5), (31316, 5), (33538, 5), (33457, 5), (33313, 5), (28494, 5)]


In [9]:
DEBUG_LIMIT = None
offset = 1
xs = []
ys = []
for worm_idx in range(len(list_of_curve_files)):
    if(worm_idx == worm_idx): 
        muscle_points = list_of_muscle_points[worm_idx]
        curve_points = list_of_curve_points[worm_idx]
        print("Curve point size: " + str(len(curve_points)) + " Muscle point size: " + str(len(muscle_points)))

        for idx in range(len(curve_points[:DEBUG_LIMIT])- offset):
            y = muscle_points[idx+ int(offset/2)+1]
            ys.append(y)
            x = np.array([])
            for off in range(offset+1):
                x = np.concatenate((x, curve_points[idx+off]))
            xs.append(x)

ys = np.array(ys)[0:]
xs = np.array(xs)[0:]
print(ys.shape)
print(xs.shape)

Curve point size: 29871 Muscle point size: 29871
Curve point size: 31844 Muscle point size: 31844
Curve point size: 31862 Muscle point size: 31862
Curve point size: 32980 Muscle point size: 32980
Curve point size: 33095 Muscle point size: 33095
Curve point size: 33166 Muscle point size: 33166
Curve point size: 31316 Muscle point size: 31316
Curve point size: 33538 Muscle point size: 33538
Curve point size: 33457 Muscle point size: 33457
Curve point size: 33313 Muscle point size: 33313
Curve point size: 28494 Muscle point size: 28494
(352925, 5)
(352925, 10)


In [10]:
# Normalise
xs_norm = np.nan_to_num((xs - xs.min(axis=0)) / (xs.max(axis=0)- xs.min(axis=0)))
ys_norm = np.nan_to_num((ys - ys.min(axis=0)) / (ys.max(axis=0)- ys.min(axis=0)))

print(xs_norm.shape)
print(ys_norm.shape)
print("\n")
print("X Min: " + str(np.amin(xs_norm)) + " X Max: " + str(np.amax(xs_norm)))
print("Y Min: " + str(np.amin(ys_norm)) + " Y Max: " + str(np.amax(ys_norm)))

(352925, 10)
(352925, 5)


X Min: 0.0 X Max: 1.0
Y Min: 0.0 Y Max: 1.0


In [11]:
print("X Min: " + str(np.amin(xs)) + " X Max: " + str(np.amax(xs)))
print("Y Min: " + str(np.amin(ys)) + " Y Max: " + str(np.amax(ys)))
original_y_min = np.amin(ys)
original_y_max = np.amax(ys)
original_x_min = np.amin(xs)
original_x_max = np.amax(xs)

X Min: -13.6624 X Max: 13.4188
Y Min: -24.908524 Y Max: 33.532132


In [12]:
 # LOAD IN NEW DATA HERE
def NEW_inner_extract_x_only_vector(list_of_curve_points, worm_idx,xs):
    DEBUG_LIMIT = None
    offset = 1
    curve_points = list_of_curve_points[worm_idx]
    for idx in range(len(curve_points[:DEBUG_LIMIT])- offset):
        x = np.array([])
        for off in range(offset+1):
            x = np.concatenate((x, curve_points[idx+off]))
        xs.append(x)
    return np.array(xs)
def NEW_inner_extract_xy_vector(list_of_muscle_points, list_of_curve_points, worm_idx, xs, ys):
    DEBUG_LIMIT = None
    offset = 1
    muscle_points = list_of_muscle_points[worm_idx]
    curve_points = list_of_curve_points[worm_idx]
    print("Curve point size: " + str(len(curve_points)) + " Muscle point size: " + str(len(muscle_points)))
    
    for idx in range(len(curve_points[:DEBUG_LIMIT])- offset):
        y = muscle_points[idx+ int(offset/2)+1]
        ys.append(y)
        x = np.array([])
        for off in range(offset+1):
            x = np.concatenate((x, curve_points[idx+off]))
        xs.append(x)
    return np.array(xs), np.array(ys)

In [14]:
E_VALUE="e0p1"
new_x_data = np.loadtxt(open("kappa_unseen_data/kappa_unseen_data_"+ E_VALUE + ".txt", "rb"), delimiter=",")
new_y_data = np.loadtxt(open("beta_unseen_data/eigvals0.txt", "rb"), delimiter=",")

In [15]:
#Create the list of files
new_x_data_list = [new_x_data]
new_y_data_list = [new_y_data]
n_xs ,n_ys = NEW_inner_extract_xy_vector(new_y_data_list, new_x_data_list, 0, [], [])

Curve point size: 33486 Muscle point size: 33486


In [16]:
print("(Unseen Generated Kappa) Xs Shape: " + str(n_xs.shape) + ",Xs Min: " + str(np.amin(n_xs)) + ", Xs Max: " + str(np.amax(n_xs)))
print("(Unseen Fake Beta) Ys Shape: " + str(n_ys.shape) + ", Ys Min: " + str(np.amin(n_ys)) + ", Ys Max: " + str(np.amax(n_ys)))

(Unseen Generated Kappa) Xs Shape: (33485, 10),Xs Min: -14.0341, Xs Max: 13.2323
(Unseen Fake Beta) Ys Shape: (33485, 5), Ys Min: -19.650889, Ys Max: 27.380742


In [17]:
 # Normalise new data to within bounds
# n_xs_norm = xs_norm
# n_ys_norm = ys_norm
n_xs_norm = np.nan_to_num((n_xs - original_x_min) / (original_x_max- original_x_min))
n_ys_norm = np.nan_to_num((n_ys - original_y_min) / (original_y_max- original_y_min))
# n_xs_norm = np.nan_to_num((n_xs - n_xs.min(axis=0)) / (n_xs.max(axis=0)- n_xs.min(axis=0)))
# n_ys_norm = np.nan_to_num((n_ys - n_ys.min(axis=0)) / (n_ys.max(axis=0)- n_ys.min(axis=0)))
print("n_Xs Shape: " + str(n_xs_norm.shape) + ", n_Xs Min: " + str(np.amin(n_xs_norm)) + ", n_Xs Max: " + str(np.amax(n_xs_norm)))
print("n_Ys Shape: " + str(n_ys_norm.shape) + ", n_Ys Min: " + str(np.amin(n_ys_norm)) + ", n_Ys Max: " + str(np.amax(n_ys_norm)))

n_Xs Shape: (33485, 10), n_Xs Min: -0.013725388830627912, n_Xs Max: 0.9931133036940757
n_Ys Shape: (33485, 5), n_Ys Min: 0.08996536589185448, n_Ys Max: 0.8947412568401012


In [ ]:
 # Generate a quick random string (prevents file save bug)
np.random.seed()
rand_str = str(np.random.randint(1000))
# Fix random seed for reproducibility
np.random.seed(7)
timestep = 5
# (nb_samples, timesteps, input_dim)
xs_norm_rec, ys_norm_rec = MlModelDataFormat.convert_data_set_to_recurrence(n_xs_norm, n_ys_norm, timestep)
print(xs_norm_rec.shape)
print(ys_norm_rec.shape)

In [ ]:
np.random.seed()
rand_str = str(np.random.randint(1000))

# Fix random seed for reproducibility
np.random.seed(7)

# Data shape
num_train_samples = int(xs_norm.shape[0] * 0.8)

timestep = 5

def split_into_train_test(x, y, num_train_samples):
    # Split into train/ test sets
    num_test_samples = x.shape[0] - num_train_samples

    print("Number Test Samples: " + str(num_test_samples))
    trainX = x[0:num_train_samples,:,:]
    trainY = y[0:num_train_samples,:]

    testX = x[num_train_samples:num_train_samples+num_test_samples,:,:]
    testY = y[num_train_samples:num_train_samples+num_test_samples,:]
    return trainX, trainY, testX, testY


xs_norm_rec, ys_norm_rec = MlModelDataFormat.convert_data_set_to_recurrence(xs_norm, ys_norm, timestep)

trainX, trainY, testX, testY = split_into_train_test(xs_norm_rec, ys_norm_rec, num_train_samples)

# Check the output shapes
print("Final shapes: train_x=" + str(trainX.shape) + ", train_y=" 
      + str(trainY.shape) + ", test_x=" + str(testX.shape) + ", test_y=" +str(testY.shape))
print(ys_norm_rec[1290:1295])
print(testY[90:95])

In [ ]:
model = k.models.Sequential()
model.add(k.layers.LSTM(20, input_shape=(timestep,10), return_sequences=True))
model.add(k.layers.LSTM(5, input_shape=(timestep,10), return_sequences=False))
model.add(k.layers.Dense(5))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=["mape"])
history = model.fit(trainX, trainY, validation_split=0.2, epochs=100, batch_size=100, verbose=2)

try:
    model.save_weights('neural-1-unit-weights-v2-e10-base'+
                       str(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))+rand_str+'.h5')
except OSError:
    print("Unable to save weights.")

In [ ]:
MlPlots.plot_model_log_loss(history)
MlPlots.plot_model_loss(history)

In [ ]:
print(model.summary())

In [ ]:
def plot_coefficient_reconstruction(model, X, Y):
    plt.clf()
    fig, axarr = plt.subplots(2, 2)
    for coeff_idx in range(4):
        if(coeff_idx == 0):
            i, j = 0,0
        elif(coeff_idx == 1):
            i, j = 0,1
        elif(coeff_idx == 2):
            i, j = 1,0
        elif(coeff_idx == 3):
            i, j = 1,1
        else:
            break
        MAX = 100000

        prediction_raw_sample = model.predict(X)[0:MAX][:,coeff_idx]
        Y_raw_sample = Y[0:MAX][:,coeff_idx]
        predictions = prediction_raw_sample
        
        axarr[i,j].plot((predictions), label="Predicted", c="red", linewidth=0.8)
        axarr[i,j].plot(Y_raw_sample[:], label="Data", linewidth=0.8)
        axarr[i,j].title.set_text("Mode " + str(coeff_idx+1))
        axarr[i,j].set_ylim([0,1])
        axarr[i,j].set_xlim([0,length])
        axarr[i,j].set_xlabel("Frame Number (16FPS)")
        axarr[i,j].set_ylabel("Coefficient")
        axarr[i,j].axvline(x=start_of_test, ymin=0, ymax=100, linewidth=1, color='k')
        axarr[i,j].legend(fontsize=10,bbox_to_anchor=(1, 1),loc='center')

    plt.rcParams['figure.figsize'] = 10, 5
    plt.tight_layout(pad=3.0, w_pad=5.5, h_pad=1.0)
    plt.suptitle("Eigemode Reconstruction")
    plt.show()

X, Y = MlPlots.switch_test_train(trainX, trainY, testX, testY, PlotType.test)
plot_coefficient_reconstruction(model, X[start:start+length], Y[start:start+length])